In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

def fetch_binance_klines(symbol, interval, limit, end_time_ms=None):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
    }
    if end_time_ms:
        params["endTime"] = end_time_ms

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return []

def download_last_2500_1m(symbol="BNBUSDT", interval="1m", file="BNBUSDT_last_2500.csv"):
    all_data = []

    # Step 1: Fetch latest 1000 candles
    data_1 = fetch_binance_klines(symbol, interval, 1000)
    if not data_1:
        print("🚫 Failed to fetch latest 1000 candles.")
        return
    all_data.extend(data_1)

    # Step 2: Fetch previous 1000 candles
    end_time_2 = int(data_1[0][0]) - 1
    data_2 = fetch_binance_klines(symbol, interval, 1000, end_time_ms=end_time_2)
    if not data_2:
        print("🚫 Failed to fetch previous 1000 candles.")
        return
    all_data = data_2 + all_data

    # Step 3: Fetch previous 500 candles before those
    end_time_3 = int(data_2[0][0]) - 1
    data_3 = fetch_binance_klines(symbol, interval, 500, end_time_ms=end_time_3)
    if not data_3:
        print("🚫 Failed to fetch previous 500 candles.")
        return
    all_data = data_3 + all_data

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')

    df.to_csv(file, index=False)
    print(f"📁 Saved {len(df)} rows to {file}")

# 🔧 Run this
download_last_2500_1m()


📁 Saved 2500 rows to BNBUSDT_last_2500.csv


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("BNBUSDT_1m_6months.csv", parse_dates=["timestamp"], index_col="timestamp")
df = df.sort_index()

for col in ["open", "high", "low", "close"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")



# HAC
df["HA_close"] = (df["open"] + df["high"] + df["low"] + df["close"]) / 4
df["HA_open"] = ((df["open"].shift(1) + df["close"].shift(1)) / 2).fillna(df["close"])

# momentum
df["momentum_10"] = df["close"] - df["close"].shift(10)
df["return_1m"] = df["close"].pct_change()

# bbwr
rolling_mean = df["close"].rolling(window=20).mean()
rolling_std = df["close"].rolling(window=20).std()
df["bb_upper"] = rolling_mean + 2 * rolling_std
df["bb_lower"] = rolling_mean - 2 * rolling_std
df["bb_width"] = df["bb_upper"] - df["bb_lower"]
df["bb_width_ratio"] = df["bb_width"] / rolling_mean


df["future_return"] = df["close"].shift(-5) / df["close"] - 1

df["label"] = np.select(
    [
        df["future_return"] > 0.002,    
        df["future_return"] < -0.002    
    ],
    [
        "Buy", "Sell"
    ],
    default="Hold"
)

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# tts
features = ["HA_close", "HA_open", "momentum_10", "return_1m", "bb_width_ratio"]
X = df[features]
y = df["label"]

# label encoding 
le = LabelEncoder()
y_encoded = le.fit_transform(y)

split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y_encoded[:split_idx], y_encoded[split_idx:]

model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)

print(" Classification Report (Buy/Sell/Hold):")
print(classification_report(y_test_labels, y_pred_labels))

print(" Confusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred_labels, labels=["Buy", "Sell", "Hold"]))

c:\Users\anura\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:43:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


 Classification Report (Buy/Sell/Hold):
              precision    recall  f1-score   support

         Buy       0.06      0.01      0.02       502
        Hold       0.91      0.89      0.90     10758
        Sell       0.05      0.10      0.06       536

    accuracy                           0.82     11796
   macro avg       0.34      0.34      0.33     11796
weighted avg       0.84      0.82      0.83     11796

 Confusion Matrix:
[[   7   31  464]
 [  18   52  466]
 [  95 1035 9628]]
